In [ ]:
import sys,os
import scipy.io as spio
import numpy as np
from quspin.tools.measurements import obs_vs_time
import pickle
from quspin.operators import hamiltonian
from quspin.operators import quantum_LinearOperator
from quspin.basis import boson_basis_1d # bosonic Hilbert space 
import time
import matplotlib.pyplot as plt # plotting library
from quspin.tools.Floquet import Floquet_t_vec # stroboscopic time vector

from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d, spinless_fermion_basis_1d # Hilbert spacespin basis
import numpy as np # generic math functions
import matplotlib.pyplot as plt # plotting library
from quspin.tools.measurements import ent_entropy, diag_ensemble # entropies
dtype_real = np.float64
dtype_cmplx = np.result_type(dtype_real,np.complex64)
from matplotlib import ticker, cm 
import tqdm

def parallel_QFI(Omega,L,Nsteps,t):
    import numpy as np
    
    dw = Omega / 1e+3
    L=L # system size
    h=1. # z magnetic field strength
    J=0.1 * h # spin xx interaction
    g=0
    scale = 1/h
    "time_evolution until 100 period"
    #tf = 5.
    Nsteps = Nsteps
    t=t
    #t=scale * np.linspace(0,tf,Nsteps)
    dt = tf/Nsteps
    print(dt*scale)

    H,psi_i= H_Ising(Omega,L,h,g,J)
    Hp,psi_i = H_Ising(Omega+dw,L,h,g,J)
    Hm,psi_i = H_Ising(Omega-dw,L,h,g,J)
    H2p,psi_i = H_Ising(Omega+2*dw,L,h,g,J)
    H2m,psi_i = H_Ising(Omega-2*dw,L,h,g,J)

    psit=H.evolve(psi_i,0,t)
    psitp=Hp.evolve(psi_i,0,t)
    psitm=Hm.evolve(psi_i,0,t)
    psit2p=H2p.evolve(psi_i,0,t)
    psit2m=H2p.evolve(psi_i,0,t)
    
    QFI = np.zeros(Nsteps)
    for ii in range(Nsteps):
        diff     = (-psit2p[:,ii] + 8*psitp[:,ii] - 8*psitm[:,ii] + psit2m[:,ii]) / (12 * dw)

        QFI[ii]      = diff.transpose().conj().dot(diff) - abs(psit[:,ii].transpose().conj().dot(diff))
    
    #print(np.real(QFI))  
    return np.real(QFI)




def H_Ising(Omega,L,h,g,J):
    
    import numpy as np
    
    def drive1(t,Omega):
        return np.cos(2.*Omega*t)
    drive1_args=[Omega]
    
    def drive2(t,Omega):
        return np.cos(Omega*t)
    drive2_args=[Omega]
    

    "define site-coupling lists for operators"
    x_field=[[g,i] for i in range(L)]
    z_field=[[h,i] for i in range(L)]
    z_field2 = []
    for ii in range(L):
        if ii== L-1:
            z_field2.append([h,ii])
        else:
            z_field2.append([h,ii])
            
    J_nn = []
    for ii in range(L-1):
        J_nn.append([J,ii,ii+1])
    "Floquet DD"
    J_nn1 = []
    for ii in range(L-1):
        if ii%2==0:
            J_nn1.append([J,ii,ii+1])
        else:
            J_nn1.append([0.,ii,ii+1])
    J_nn2 = []
    for ii in range(L-1):
        if ii%2==1:
            J_nn2.append([J,ii,ii+1])
        else:
            J_nn2.append([0.,ii,ii+1])


    dynamic1=[["xx",J_nn1,drive1,drive1_args],["xx",J_nn2,drive2,drive2_args]]
  
    dynamic2 = []

    # static and dynamic lists
    static=[["x",x_field],["z",z_field]]

    static2=[["x",x_field],["z",z_field2],["xx",J_nn]]
  

    "construct spin basis"
    basis= spin_basis_1d(L=L,S='1/2',pauli=-1)    
    ###### construct Hamiltonian
    H=hamiltonian(static,dynamic1,dtype=np.float64,basis=basis)
    H2=hamiltonian(static2,dynamic2,dtype=np.float64,basis=basis)


    "Ground State of H2" 
    E,V=H2.eigsh(time=0.0,k=1,which='SA')
    psi_i=V[:,0]
    
    return H, psi_i


N=3
Nvec = 100
h=1.
w_vec = h* np.linspace(0., 6, Nvec)
tf = 1000.
Nsteps = 100000
t=np.linspace(0,tf,Nsteps)
QFI = np.zeros((Nsteps,Nvec))

for zz in tqdm.tqdm(range(Nvec)):
       QFI[:,zz] = parallel_QFI(w_vec[zz],N,Nsteps,t)

results = [w_vec, t, QFI]
with open("QFI-QuSpinL3.pickle",'wb') as handle:
    pickle.dump(results, handle) 


fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)
#"3D plot auto-correlation"
X, Y = np.meshgrid(w_vec,t)


cp = plt.contourf(X,Y,QFI,levels=30,cmap=cm.inferno)
    # plt.text(3., 45, r'(d)',{'color': 'black', 'fontsize': 40, 'ha': 'center', 'va': 'center'}, style='normal')

ax.xaxis.set_major_locator(MultipleLocator(1.0))
#ax.xaxis.set_minor_locator(MultipleLocator(1.0))
ax.yaxis.set_major_locator(MultipleLocator(100))
ax.yaxis.set_minor_locator(MultipleLocator(50))
#ax.tick_params(which='both', width=1.0,direction='out', top='off',right='off')
ax.tick_params(which='major', width=1.0, length=10,direction='out')
ax.tick_params(which='minor', length=8, color='k',direction='out')


plt.ylabel('$t/T$')
plt.xlabel('Bond')
plt.colorbar().set_label(r'Concurrence')
plt.savefig("QFI.png", format="png")
plt.show()

#plt.figure(figsize=(6,4))
#X, Y = np.meshgrid(w_vec,t)
#Z = QFI
#cs = plt.contourf(X, Y, Z,levels=30, 
 #                 locator = ticker.LogLocator(), 
 #                 cmap ="viridis") 
  
#cbar = plt.colorbar(cs) 
  
#plt.title('matplotlib.pyplot.contourf() Example') 
#plt.show() 

plt.figure(figsize=(6,4))
plt.plot(w_vec,QFI[:,199],'o-',       label = "L = " + str(N))
plt.legend()
plt.ylabel("Quantum Fisher Information")
plt.xlabel(r"$J_2$")

ax = plt.gca()
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.show()


